In [1]:
import pandas as pd
# from dowhy import gcm
import numpy as np
import matplotlib.pyplot as plt
import gc

import os

In [2]:
#import exp_filtered
data = pd.read_csv('./Extraction/experiments_filtered.csv',header=0, index_col=0)
experiments = pd.DataFrame(data)

experiments.tail(10)


,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
288,745,2024-01-19 20:52:52.8522174,2024-01-19 21:22:52.8564323,general,random27,8,1,23,4,8,2,0.25,3.0,1705697572,1705699372
289,746,2024-01-19 21:26:27.3234593,2024-01-19 21:56:27.3265222,general,random27,8,1,23,4,8,2,0.25,3.0,1705699587,1705701387
290,747,2024-01-19 22:00:01.6380175,2024-01-19 22:30:01.6457071,general,random28,20,18,23,8,4,16,0.50,1.0,1705701601,1705703401
294,751,2024-01-20 00:14:24.4097982,2024-01-20 00:44:24.4126883,general,random29,18,20,8,4,6,4,0.25,3.0,1705709664,1705711464
295,752,2024-01-20 00:47:58.7269655,2024-01-20 01:17:58.7312773,general,random29,18,20,8,4,6,4,0.25,3.0,1705711678,1705713478
296,753,2024-01-20 01:21:32.6299318,2024-01-20 01:51:32.6406703,general,random30,15,4,20,8,2,8,1.00,1.0,1705713692,1705715492
299,758,2024-01-20 04:09:25.0644731,2024-01-20 04:39:25.0711445,general,random31,8,6,18,4,4,4,0.25,0.5,1705723765,1705725565
301,760,2024-01-20 05:16:34.1638368,2024-01-20 05:46:34.1666149,general,random32,3,2,5,2,2,2,0.50,1.0,1705727794,1705729594
302,761,2024-01-20 05:50:08.2461471,2024-01-20 06:20:08.2541699,general,random32,3,2,5,2,2,2,0.50,1.0,1705729808,1705731608
303,762,2024-01-20 06:23:42.6007382,2024-01-20 06:53:42.6043583,general,random33,19,8,23,4,8,2,0.25,3.0,1705731822,1705733622


In [3]:
messages = pd.DataFrame()

for index, row in experiments.iterrows():
    data = pd.read_csv('../../datasets/partialTables/mess_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_mess = pd.DataFrame(data)
    query_mess = query_mess.drop(columns=["start", "end"])

    messages = pd.concat([query_mess, messages])
    del query_mess
    gc.collect()

    if os.path.exists('../../datasets/partialTables/ihave_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../../datasets/partialTables/ihave_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_ihave = pd.DataFrame(data)
        query_ihave["type"] = '17.0'
        query_ihave = query_ihave.drop(columns=["_measurement"])
        
        messages = pd.concat([query_ihave, messages])
        del query_ihave
        gc.collect()
    
    if os.path.exists('../../datasets/partialTables/iwant_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../../datasets/partialTables/iwant_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_iwant = pd.DataFrame(data)
        query_iwant["type"] = '16.0'
        query_iwant = query_iwant.drop(columns=["_measurement"])
    
        messages = pd.concat([query_iwant, messages])
        del query_iwant
        gc.collect()
    

messages.head(10)

,_time,identifier,experiment,type
0,2024-01-20 06:31:00.348127+00:00,303,762,16.0
1,2024-01-20 06:46:45.354625+00:00,303,762,16.0
2,2024-01-20 06:46:45.355063+00:00,303,762,16.0
3,2024-01-20 06:46:57.349052+00:00,303,762,16.0
0,2024-01-20 06:23:43.925455+00:00,303,762,17.0
1,2024-01-20 06:23:43.967483+00:00,303,762,17.0
2,2024-01-20 06:23:43.999768+00:00,303,762,17.0
3,2024-01-20 06:23:43.999801+00:00,303,762,17.0
4,2024-01-20 06:23:44.000139+00:00,303,762,17.0
5,2024-01-20 06:23:44.005508+00:00,303,762,17.0


In [4]:
# Transpose the table -> type and count
df = messages.drop(columns=['identifier'])
df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],df['type'],pd.Grouper(key="_time", freq='10s')])['index'].count().reset_index()


pivoted = pd.pivot_table(by_time,index=["experiment", pd.Grouper(freq="1s", key="_time")], columns="type", values="index", margins=False).fillna(0)
pivoted = pivoted.reset_index().rename_axis(None, axis=1)

pivoted.head(100)

/tmp/ipykernel_504652/1170785966.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,2.0,3.0,11.0,12.0,16.0,17.0
0,390,2023-10-04 18:52:00+00:00,8684.0,1193.0,0.0,0.0,0.0,698.0
1,390,2023-10-04 18:52:10+00:00,16138.0,2188.0,0.0,0.0,0.0,1032.0
2,390,2023-10-04 18:52:20+00:00,13689.0,1874.0,0.0,0.0,0.0,1339.0
3,390,2023-10-04 18:52:30+00:00,13750.0,1898.0,0.0,0.0,0.0,963.0
4,390,2023-10-04 18:52:40+00:00,13993.0,1930.0,0.0,0.0,2.0,968.0
...,...,...,...,...,...,...,...,...
95,390,2023-10-04 19:11:20+00:00,24258.0,3292.0,0.0,0.0,1.0,1466.0
96,390,2023-10-04 19:11:30+00:00,19123.0,2605.0,0.0,0.0,0.0,1243.0
97,390,2023-10-04 19:11:40+00:00,19142.0,2603.0,0.0,0.0,0.0,1477.0
98,390,2023-10-04 19:11:50+00:00,23490.0,3204.0,0.0,0.0,0.0,1444.0


In [5]:
#Rename
eventsTS = pivoted.rename(columns={2.0:'messageDuplicated', 3.0: 'messageReceived', 11.0: 'graft', 12.0: 'prune', '16.0': 'iwant', '17.0':'ihave'})

eventsTS.to_csv('./events10s.csv')
eventsTS.head(500)

,experiment,_time,messageDuplicated,messageReceived,graft,prune,iwant,ihave
0,390,2023-10-04 18:52:00+00:00,8684.0,1193.0,0.0,0.0,0.0,698.0
1,390,2023-10-04 18:52:10+00:00,16138.0,2188.0,0.0,0.0,0.0,1032.0
2,390,2023-10-04 18:52:20+00:00,13689.0,1874.0,0.0,0.0,0.0,1339.0
3,390,2023-10-04 18:52:30+00:00,13750.0,1898.0,0.0,0.0,0.0,963.0
4,390,2023-10-04 18:52:40+00:00,13993.0,1930.0,0.0,0.0,2.0,968.0
...,...,...,...,...,...,...,...,...
495,394,2023-10-04 21:08:50+00:00,6677.0,666.0,0.0,0.0,0.0,0.0
496,394,2023-10-04 21:09:00+00:00,8460.0,847.0,0.0,0.0,0.0,0.0
497,394,2023-10-04 21:09:10+00:00,6772.0,680.0,0.0,0.0,0.0,0.0
498,394,2023-10-04 21:09:20+00:00,6665.0,667.0,0.0,0.0,0.0,0.0


In [6]:
#Import partial tables and append -> for propagation time

publishMessage = pd.DataFrame()
deliverMessage = pd.DataFrame()

for index, row in experiments.iterrows():
    data = pd.read_csv('../../datasets/partialTables/mess2_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_mess = pd.DataFrame(data)

    data = pd.read_csv('../../datasets/partialTables/pub_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_pub = pd.DataFrame(data)
    
    deliverMessage = pd.concat([query_mess, deliverMessage])
    publishMessage = pd.concat([publishMessage, query_pub])

    del query_pub
    del query_mess

    gc.collect()

deliverMessage.head(10)


,_time,messageID,experiment
0,2024-01-20 06:23:42.139365+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
1,2024-01-20 06:23:42.139457+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
2,2024-01-20 06:23:42.139631+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
3,2024-01-20 06:23:42.140159+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
4,2024-01-20 06:23:42.140332+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
5,2024-01-20 06:23:42.140430+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
6,2024-01-20 06:23:42.140436+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
7,2024-01-20 06:23:42.141264+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
8,2024-01-20 06:23:42.141383+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
9,2024-01-20 06:23:42.141725+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762


In [7]:
publishMessage['_time'] = pd.to_datetime(publishMessage["_time"], format='mixed')#.tz_localize(None)
deliverMessage['_time'] = pd.to_datetime(deliverMessage["_time"], format='mixed')#.tz_localize(None)

# def calcAverageTime(publish, received, expTime, parameter):
expTime = experiments
publish = publishMessage
received = deliverMessage

publish = publish[['_time', 'messageID','experiment']]
received = received[['_time', 'messageID', 'experiment']]

joined = publish.merge(received, on=['messageID', 'experiment'])
joined['diff'] = ((joined['_time_y'] - joined['_time_x'])/ pd.Timedelta(microseconds=1)).astype(int)
joined = joined.loc[joined["diff"] >= 0].dropna()
joined.head(10)


,_time_x,messageID,experiment,_time_y,diff
0,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.134903+00:00,753
1,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.136298+00:00,2148
2,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.136886+00:00,2736
3,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137090+00:00,2940
4,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137205+00:00,3055
5,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137276+00:00,3126
6,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137353+00:00,3203
7,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137377+00:00,3227
8,2023-10-04 18:52:04.134667+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.134975+00:00,308
9,2023-10-04 18:52:04.134667+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.135169+00:00,502


In [8]:
df = joined.drop(columns=['_time_x', 'messageID']).rename(columns={'_time_y' : '_time'})

df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],pd.Grouper(key="_time", freq='10s')])['diff'].agg('mean').reset_index()
by_time.head(100)

/tmp/ipykernel_504652/4286236444.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,diff
0,390,2023-10-04 18:52:00+00:00,1773.489130
1,390,2023-10-04 18:52:10+00:00,2037.555657
2,390,2023-10-04 18:52:20+00:00,1770.802686
3,390,2023-10-04 18:52:30+00:00,1765.751992
4,390,2023-10-04 18:52:40+00:00,1960.033882
...,...,...,...
95,390,2023-10-04 19:11:20+00:00,2009.049621
96,390,2023-10-04 19:11:30+00:00,2122.233244
97,390,2023-10-04 19:11:40+00:00,2088.376773
98,390,2023-10-04 19:11:50+00:00,1900.575295


In [9]:
prop = by_time.rename(columns={'diff':'propagationTime'})

prop = prop.merge(eventsTS, on=['experiment', '_time'], how='outer')
prop.head(100)

,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave
0,390,2023-10-04 18:52:00+00:00,1773.489130,8684.0,1193.0,0.0,0.0,0.0,698.0
1,390,2023-10-04 18:52:10+00:00,2037.555657,16138.0,2188.0,0.0,0.0,0.0,1032.0
2,390,2023-10-04 18:52:20+00:00,1770.802686,13689.0,1874.0,0.0,0.0,0.0,1339.0
3,390,2023-10-04 18:52:30+00:00,1765.751992,13750.0,1898.0,0.0,0.0,0.0,963.0
4,390,2023-10-04 18:52:40+00:00,1960.033882,13993.0,1930.0,0.0,0.0,2.0,968.0
...,...,...,...,...,...,...,...,...,...
95,390,2023-10-04 19:11:20+00:00,2009.049621,24258.0,3292.0,0.0,0.0,1.0,1466.0
96,390,2023-10-04 19:11:30+00:00,2122.233244,19123.0,2605.0,0.0,0.0,0.0,1243.0
97,390,2023-10-04 19:11:40+00:00,2088.376773,19142.0,2603.0,0.0,0.0,0.0,1477.0
98,390,2023-10-04 19:11:50+00:00,1900.575295,23490.0,3204.0,0.0,0.0,0.0,1444.0


In [10]:
prop.to_csv('./propagationTime10s.csv')

In [11]:
#Bandwidth

#Load rpc dataset
# data = pd.read_csv('./Extraction/rpc.csv',header=0, index_col=0)
# rpc = pd.DataFrame(data)
# rpc = rpc.rename(columns={'rpc':'count'})

# rpc.head(10)

rpc = pd.DataFrame()


for idx, exec in experiments.iterrows():
    data = pd.read_csv('../../datasets/partialTables/rpc_'+str(exec['experiment'])+'.csv', header=0,  index_col=0)
    query_rpc = pd.DataFrame(data)
        
    rpc = pd.concat([rpc,query_rpc])

    del query_rpc
    gc.collect()
        

rpc.head(10)

,_time,_measurement,experiment,start,end
0,2023-10-04 18:52:04.134925+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
1,2023-10-04 18:52:04.135293+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
2,2023-10-04 18:52:04.135840+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
3,2023-10-04 18:52:04.135852+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
4,2023-10-04 18:52:04.135928+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
5,2023-10-04 18:52:04.135930+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
6,2023-10-04 18:52:04.135938+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
7,2023-10-04 18:52:04.135962+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
8,2023-10-04 18:52:04.135988+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
9,2023-10-04 18:52:04.136030+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518


In [12]:
rpc = rpc.drop(columns=['start', 'end'])
                       
rpc.reset_index(inplace=True)
rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)
by_time = rpc.groupby([rpc['experiment'],pd.Grouper(key="_time", freq='10s')])['index'].agg('count').reset_index()
# by_time.head(100)

rpc=by_time.rename(columns={'index':'rpc'})

/tmp/ipykernel_504652/564874031.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)


In [13]:
rpc.head(10)

,experiment,_time,rpc
0,390,2023-10-04 18:52:00+00:00,10149
1,390,2023-10-04 18:52:10+00:00,18719
2,390,2023-10-04 18:52:20+00:00,16109
3,390,2023-10-04 18:52:30+00:00,15989
4,390,2023-10-04 18:52:40+00:00,16295
5,390,2023-10-04 18:52:50+00:00,18862
6,390,2023-10-04 18:53:00+00:00,16217
7,390,2023-10-04 18:53:10+00:00,16887
8,390,2023-10-04 18:53:20+00:00,18650
9,390,2023-10-04 18:53:30+00:00,17171


In [14]:
#Total bandwidth

# data = pd.read_csv('../datasets/timeSeries1s.csv', header=0,  index_col=0)
# message = pd.DataFrame(data)

# idx = [3.0]
# message = message[message["type"].isin(idx)]
# message = message.drop(columns=["type", "identifier"])

message=received

message.reset_index(inplace=True)
message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)
by_time = message.groupby([message['experiment'],pd.Grouper(key="_time", freq='10s')])['messageID'].agg('count').reset_index()
message = by_time

message.head(10)

/tmp/ipykernel_504652/3772719910.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,messageID
0,390,2023-10-04 18:52:00+00:00,1199
1,390,2023-10-04 18:52:10+00:00,2200
2,390,2023-10-04 18:52:20+00:00,1882
3,390,2023-10-04 18:52:30+00:00,1909
4,390,2023-10-04 18:52:40+00:00,1936
5,390,2023-10-04 18:52:50+00:00,2221
6,390,2023-10-04 18:53:00+00:00,1935
7,390,2023-10-04 18:53:10+00:00,2009
8,390,2023-10-04 18:53:20+00:00,2190
9,390,2023-10-04 18:53:30+00:00,2046


In [15]:
message = message.rename(columns={'messageID':'count'})
rpc = rpc.rename(columns={'rpc': 'count'})
total_bandwidth = pd.concat([rpc, message])

total_bandwidth.reset_index(inplace=True)
total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)
by_time = total_bandwidth.groupby([total_bandwidth['experiment'],pd.Grouper(key="_time", freq='10s')])['count'].agg('sum').reset_index()
by_time.head(100)

/tmp/ipykernel_504652/203799029.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,count
0,390,2023-10-04 18:52:00+00:00,11348
1,390,2023-10-04 18:52:10+00:00,20919
2,390,2023-10-04 18:52:20+00:00,17991
3,390,2023-10-04 18:52:30+00:00,17898
4,390,2023-10-04 18:52:40+00:00,18231
...,...,...,...
95,390,2023-10-04 19:11:20+00:00,31426
96,390,2023-10-04 19:11:30+00:00,24832
97,390,2023-10-04 19:11:40+00:00,24999
98,390,2023-10-04 19:11:50+00:00,30474


In [16]:
by_time['totalBandwidth'] = by_time['count']

totalBandwidth = by_time.drop(columns=['count'])
totalBandwidth.head(10)

,experiment,_time,totalBandwidth
0,390,2023-10-04 18:52:00+00:00,11348
1,390,2023-10-04 18:52:10+00:00,20919
2,390,2023-10-04 18:52:20+00:00,17991
3,390,2023-10-04 18:52:30+00:00,17898
4,390,2023-10-04 18:52:40+00:00,18231
5,390,2023-10-04 18:52:50+00:00,21083
6,390,2023-10-04 18:53:00+00:00,18152
7,390,2023-10-04 18:53:10+00:00,18896
8,390,2023-10-04 18:53:20+00:00,20840
9,390,2023-10-04 18:53:30+00:00,19217


In [17]:
#gossip bandwidth
gossipBandwidth = rpc
gossipBandwidth['gossipBandwidth'] = gossipBandwidth['count']/10
gossipBandwidth = gossipBandwidth.drop(columns=['count'])

gossipBandwidth.head(100)

,experiment,_time,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,1014.9
1,390,2023-10-04 18:52:10+00:00,1871.9
2,390,2023-10-04 18:52:20+00:00,1610.9
3,390,2023-10-04 18:52:30+00:00,1598.9
4,390,2023-10-04 18:52:40+00:00,1629.5
...,...,...,...
95,390,2023-10-04 19:11:20+00:00,2811.2
96,390,2023-10-04 19:11:30+00:00,2221.7
97,390,2023-10-04 19:11:40+00:00,2238.4
98,390,2023-10-04 19:11:50+00:00,2724.9


In [18]:
#MessageBandwidth

messageBandwidth = message
messageBandwidth['messageBandwidth'] = messageBandwidth['count']/10
messageBandwidth = messageBandwidth.drop(columns=['count'])

messageBandwidth.head(10)

,experiment,_time,messageBandwidth
0,390,2023-10-04 18:52:00+00:00,119.9
1,390,2023-10-04 18:52:10+00:00,220.0
2,390,2023-10-04 18:52:20+00:00,188.2
3,390,2023-10-04 18:52:30+00:00,190.9
4,390,2023-10-04 18:52:40+00:00,193.6
5,390,2023-10-04 18:52:50+00:00,222.1
6,390,2023-10-04 18:53:00+00:00,193.5
7,390,2023-10-04 18:53:10+00:00,200.9
8,390,2023-10-04 18:53:20+00:00,219.0
9,390,2023-10-04 18:53:30+00:00,204.6


In [19]:
prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)

messBand = prop.merge(messageBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

messBand.head(500)

/tmp/ipykernel_504652/2873079763.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_504652/2873079763.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth
0,390,2023-10-04 18:52:00+00:00,1773.489130,8684.0,1193.0,0.0,0.0,0.0,698.0,119.9
1,390,2023-10-04 18:52:10+00:00,2037.555657,16138.0,2188.0,0.0,0.0,0.0,1032.0,220.0
2,390,2023-10-04 18:52:20+00:00,1770.802686,13689.0,1874.0,0.0,0.0,0.0,1339.0,188.2
3,390,2023-10-04 18:52:30+00:00,1765.751992,13750.0,1898.0,0.0,0.0,0.0,963.0,190.9
4,390,2023-10-04 18:52:40+00:00,1960.033882,13993.0,1930.0,0.0,0.0,2.0,968.0,193.6
...,...,...,...,...,...,...,...,...,...,...
495,394,2023-10-04 21:09:00+00:00,1433.169643,8460.0,847.0,0.0,0.0,0.0,0.0,84.8
496,394,2023-10-04 21:09:10+00:00,1462.590361,6772.0,680.0,0.0,0.0,0.0,0.0,68.0
497,394,2023-10-04 21:09:20+00:00,1537.613636,6665.0,667.0,0.0,0.0,0.0,0.0,66.9
498,394,2023-10-04 21:09:30+00:00,1441.464286,8399.0,846.0,0.0,0.0,0.0,0.0,84.8


In [21]:
messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

totalBand = messBand.merge(totalBandwidth, on=['experiment', '_time'], how='outer').fillna(0)
totalBand['totalBandwidth'] = (totalBand['totalBandwidth']+totalBand['messageDuplicated'])/10

totalBand.head(500)

/tmp/ipykernel_504652/3948177869.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_504652/3948177869.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth
0,390,2023-10-04 18:52:00+00:00,1773.489130,8684.0,1193.0,0.0,0.0,0.0,698.0,119.9,2003.2
1,390,2023-10-04 18:52:10+00:00,2037.555657,16138.0,2188.0,0.0,0.0,0.0,1032.0,220.0,3705.7
2,390,2023-10-04 18:52:20+00:00,1770.802686,13689.0,1874.0,0.0,0.0,0.0,1339.0,188.2,3168.0
3,390,2023-10-04 18:52:30+00:00,1765.751992,13750.0,1898.0,0.0,0.0,0.0,963.0,190.9,3164.8
4,390,2023-10-04 18:52:40+00:00,1960.033882,13993.0,1930.0,0.0,0.0,2.0,968.0,193.6,3222.4
...,...,...,...,...,...,...,...,...,...,...,...
495,394,2023-10-04 21:09:00+00:00,1433.169643,8460.0,847.0,0.0,0.0,0.0,0.0,84.8,1857.2
496,394,2023-10-04 21:09:10+00:00,1462.590361,6772.0,680.0,0.0,0.0,0.0,0.0,68.0,1486.0
497,394,2023-10-04 21:09:20+00:00,1537.613636,6665.0,667.0,0.0,0.0,0.0,0.0,66.9,1462.8
498,394,2023-10-04 21:09:30+00:00,1441.464286,8399.0,846.0,0.0,0.0,0.0,0.0,84.8,1844.6


In [22]:
totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

gossipBand = totalBand.merge(gossipBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

gossipBand.head(500)

/tmp/ipykernel_504652/187803194.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_504652/187803194.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,1773.489130,8684.0,1193.0,0.0,0.0,0.0,698.0,119.9,2003.2,1014.9
1,390,2023-10-04 18:52:10+00:00,2037.555657,16138.0,2188.0,0.0,0.0,0.0,1032.0,220.0,3705.7,1871.9
2,390,2023-10-04 18:52:20+00:00,1770.802686,13689.0,1874.0,0.0,0.0,0.0,1339.0,188.2,3168.0,1610.9
3,390,2023-10-04 18:52:30+00:00,1765.751992,13750.0,1898.0,0.0,0.0,0.0,963.0,190.9,3164.8,1598.9
4,390,2023-10-04 18:52:40+00:00,1960.033882,13993.0,1930.0,0.0,0.0,2.0,968.0,193.6,3222.4,1629.5
...,...,...,...,...,...,...,...,...,...,...,...,...
495,394,2023-10-04 21:09:00+00:00,1433.169643,8460.0,847.0,0.0,0.0,0.0,0.0,84.8,1857.2,926.4
496,394,2023-10-04 21:09:10+00:00,1462.590361,6772.0,680.0,0.0,0.0,0.0,0.0,68.0,1486.0,740.8
497,394,2023-10-04 21:09:20+00:00,1537.613636,6665.0,667.0,0.0,0.0,0.0,0.0,66.9,1462.8,729.4
498,394,2023-10-04 21:09:30+00:00,1441.464286,8399.0,846.0,0.0,0.0,0.0,0.0,84.8,1844.6,919.9


In [23]:
consDiscovery = gossipBand.rename(columns={'messageDuplicated' : 'messageOverhead'})#.drop(columns=['bandwidth'])
consDiscovery.to_csv('./consDiscovery.csv')

consDiscovery.head(10)

,experiment,_time,propagationTime,messageOverhead,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,1773.489130,8684.0,1193.0,0.0,0.0,0.0,698.0,119.9,2003.2,1014.9
1,390,2023-10-04 18:52:10+00:00,2037.555657,16138.0,2188.0,0.0,0.0,0.0,1032.0,220.0,3705.7,1871.9
2,390,2023-10-04 18:52:20+00:00,1770.802686,13689.0,1874.0,0.0,0.0,0.0,1339.0,188.2,3168.0,1610.9
3,390,2023-10-04 18:52:30+00:00,1765.751992,13750.0,1898.0,0.0,0.0,0.0,963.0,190.9,3164.8,1598.9
4,390,2023-10-04 18:52:40+00:00,1960.033882,13993.0,1930.0,0.0,0.0,2.0,968.0,193.6,3222.4,1629.5
5,390,2023-10-04 18:52:50+00:00,1962.132923,16160.0,2210.0,0.0,0.0,0.0,1283.0,222.1,3724.3,1886.2
6,390,2023-10-04 18:53:00+00:00,1982.278101,13942.0,1926.0,0.0,0.0,0.0,970.0,193.5,3209.4,1621.7
7,390,2023-10-04 18:53:10+00:00,1808.053122,14533.0,2002.0,0.0,0.0,0.0,961.0,200.9,3342.9,1688.7
8,390,2023-10-04 18:53:20+00:00,1996.014559,15959.0,2179.0,0.0,0.0,0.0,1292.0,219.0,3679.9,1865.0
9,390,2023-10-04 18:53:30+00:00,1823.973827,14792.0,2039.0,0.0,0.0,0.0,957.0,204.6,3400.9,1717.1


In [25]:
consFinal = consDiscovery.merge(experiments, on='experiment')
consFinal = consFinal.drop(columns=['_time', 'start', 'end', 'startUnix', 'endUnix', 'parameter'])#, 'originalStart', 'originalEnd']) 

consFinal.to_csv('../consolidated.csv')
consFinal.head(10)


,experiment,propagationTime,messageOverhead,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipBandwidth,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval
0,390,1773.489130,8684.0,1193.0,0.0,0.0,0.0,698.0,119.9,2003.2,1014.9,unl,8,6,12,4,8,2,0.25,1.0
1,390,2037.555657,16138.0,2188.0,0.0,0.0,0.0,1032.0,220.0,3705.7,1871.9,unl,8,6,12,4,8,2,0.25,1.0
2,390,1770.802686,13689.0,1874.0,0.0,0.0,0.0,1339.0,188.2,3168.0,1610.9,unl,8,6,12,4,8,2,0.25,1.0
3,390,1765.751992,13750.0,1898.0,0.0,0.0,0.0,963.0,190.9,3164.8,1598.9,unl,8,6,12,4,8,2,0.25,1.0
4,390,1960.033882,13993.0,1930.0,0.0,0.0,2.0,968.0,193.6,3222.4,1629.5,unl,8,6,12,4,8,2,0.25,1.0
5,390,1962.132923,16160.0,2210.0,0.0,0.0,0.0,1283.0,222.1,3724.3,1886.2,unl,8,6,12,4,8,2,0.25,1.0
6,390,1982.278101,13942.0,1926.0,0.0,0.0,0.0,970.0,193.5,3209.4,1621.7,unl,8,6,12,4,8,2,0.25,1.0
7,390,1808.053122,14533.0,2002.0,0.0,0.0,0.0,961.0,200.9,3342.9,1688.7,unl,8,6,12,4,8,2,0.25,1.0
8,390,1996.014559,15959.0,2179.0,0.0,0.0,0.0,1292.0,219.0,3679.9,1865.0,unl,8,6,12,4,8,2,0.25,1.0
9,390,1823.973827,14792.0,2039.0,0.0,0.0,0.0,957.0,204.6,3400.9,1717.1,unl,8,6,12,4,8,2,0.25,1.0
